# การตรวจจับความผิดปกติ

ในแบบฝึกหัดนี้ คุณจะได้นำอัลกอริทึมการตรวจจับความผิดปกติไปใช้และนำไปใช้ในการตรวจจับเซิร์ฟเวอร์ที่ล้มเหลวบนเครือข่าย




# โครงร่าง
- [ 1 - แพ็คเกจ ](#1)
- [ 2 - การตรวจจับความผิดปกติ](#2)
  - [ 2.1 ปัญหา](#2.1)
  - [ 2.2  ชุดข้อมูล](#2.2)
  - [ 2.3 การแจกแจงแบบเกาส์เซียน](#2.3)
    - [ แบบฝึกหัด 1](#ex01)
    - [ แบบฝึกหัด 2](#ex02)
  - [ 2.4 ชุดข้อมูลมิติสูง](#2.4)


<a name="1"></a>
## 1 - แพ็กเกจ 

ก่อนอื่น มาเรียกใช้เซลล์ด้านล่างเพื่อนำเข้าแพ็กเกจทั้งหมดที่คุณจะต้องใช้ในระหว่างการกำหนดงานนี้
- [numpy](www.numpy.org)  เป็นแพ็กเกจพื้นฐานสำหรับการทำงานกับเมทริกซ์ใน Python
- [matplotlib](http://matplotlib.org) เป็นไลบรารีที่มีชื่อเสียงสำหรับการวาดกราฟใน Python
- ``utils.py`` ประกอบด้วยฟังก์ชันตัวช่วยสำหรับงานนี้นะ คุณไม่จำเป็นต้องแก้ไขโค้ดในไฟล์นี้


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:
!git clone https://github.com/Smith-WeStrideTH/Unsupervised_Learning_Course.git
%cd Unsupervised_Learning_Course/work 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils_a import *

%matplotlib inline

<a name="2"></a>
## 2 - การตรวจจับความผิดปกติ



<a name="2.1"></a>
### 2.1 คำจำกัดความปัญหา

ในแบบฝึกหัดนี้ คุณจะนำอัลกอริทึมการตรวจจับความผิดปกติมาใช้เพื่อตรวจจับพฤติกรรมผิดปกติในเซิร์ฟเวอร์คอมพิวเตอร์


ชุดข้อมูลประกอบด้วยสองคุณลักษณะ -
   * ผลผลิต (mb/s) และ
   * ละเวลาตอบสนอง (ms) ของแต่ละเซิร์ฟเวอร์

ในขณะที่เซิร์ฟเวอร์ของคุณทำงานอยู่ คุณได้รวบรวมตัวอย่าง $m=307$ ตัวอย่างของพฤติกรรมของพวกมัน ดังนั้นจึงมีชุดข้อมูลที่ไม่ได้ติดป้ายกำกับ  $\{x^{(1)}, \ldots, x^{(m)}\}$. 
* คุณสงสัยว่าตัวอย่างส่วนใหญ่เหล่านี้เป็นตัวอย่าง "ปกติ" (ไม่ผิดปกติ) ของเซิร์ฟเวอร์ที่ทำงานปกติ แต่ในชุดข้อมูลนี้อาจมีตัวอย่างบางอย่างของเซิร์ฟเวอร์ที่ทำงานผิดปกติอยู่ด้วย

คุณจะใช้แบบจำลอง Gaussian เพื่อตรวจจับตัวอย่างผิดปกติในชุดข้อมูลของคุณ
* คุณจะเริ่มต้นด้วยชุดข้อมูล 2 มิติที่จะช่วยให้คุณมองเห็นภาพว่าอัลกอริทึมกำลังทำอะไรอยู่
* บนชุดข้อมูลนั้น คุณจะปรับฟังก์ชันการแจกแจงแบบ Gaussian และค้นหาค่าที่มีความน่าจะเป็นต่ำมาก ดังนั้นจึงถือได้ว่าเป็นความผิดปกติ
* หลังจากนั้น คุณจะใช้อัลกอริทึมการตรวจจับความผิดปกติกับชุดข้อมูลขนาดใหญ่ที่มีหลายมิติ

<a name="2.2"></a>
### 2.2  ชุดข้อมูล

คุณจะเริ่มต้นด้วยการโหลดชุดข้อมูลสำหรับงานนี้
- ฟังก์ชัน  `load_data()` ที่แสดงด้านล่างจะโหลดข้อมูลลงในตัวแปร `X_train`, `X_val` และ  `y_val` 
    - คุณจะใช้ `X_train`  เพื่อปรับฟังก์ชันการแจกแจงแบบ Gaussian
    - คุณจะใช้ `X_val` และ  `y_val` เป็นชุดตรวจสอบข้ามเพื่อเลือกเกณฑ์และกำหนดตัวอย่างผิดปกติและปกติ

In [ ]:
# Load the dataset
X_train, X_val, y_val = load_data()

#### ดูตัวแปร
มาทำความคุ้นเคยกับชุดข้อมูลของคุณกัน
- วิธีที่ดีคือการพิมพ์แต่ละตัวแปรออกมาทีละตัวและดูว่ามันประกอบด้วยอะไรบ้าง

โค้ดด้านล่างจะพิมพ์ห้าองค์ประกอบแรกของแต่ละตัวแปร

In [ ]:
# Display the first five elements of X_train
print("The first 5 elements of X_train are:\n", X_train[:5])  

In [ ]:
# Display the first five elements of X_val
print("The first 5 elements of X_val are\n", X_val[:5])  

In [ ]:
# Display the first five elements of y_val
print("The first 5 elements of y_val are\n", y_val[:5])  

#### ตรวจสอบมิติของตัวแปรของคุณ

วิธีที่มีประโยชน์อีกวิธีหนึ่งในการทำความคุ้นเคยกับข้อมูลของคุณคือการดูมิติของข้อมูล

โค้ดด้านล่างจะพิมพ์รูปร่างของ `X_train`, `X_val` และ  `y_val`.

In [ ]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of X_val is:', X_val.shape)
print ('The shape of y_val is: ', y_val.shape)

#### การมองเห็นข้อมูลของคุณ

ก่อนเริ่มงานใด ๆ มักจะเป็นประโยชน์ในการทำความเข้าใจข้อมูลด้วยการมองเห็นข้อมูล
- สำหรับชุดข้อมูลนี้ คุณสามารถใช้ scatter plot เพื่อมองเห็นข้อมูล (`X_train`), เนื่องจากมีเพียงสองคุณสมบัติที่จะพล็อต (throughput และ latency)

- พล็อตของคุณควรมีลักษณะคล้ายกับด้านล่าง
<img src="images/figure1_a.png" width="500" height="500">

In [ ]:
# Create a scatter plot of the data. To change the markers to blue "x",
# we used the 'marker' and 'c' parameters
plt.scatter(X_train[:, 0], X_train[:, 1], marker='x', c='b') 

# Set the title
plt.title("The first dataset")
# Set the y-axis label
plt.ylabel('Throughput (mb/s)')
# Set the x-axis label
plt.xlabel('Latency (ms)')
# Set axis range
plt.axis([0, 30, 0, 30])
plt.show()

<a name="2.3"></a>
### 2.3 Gaussian distribution (การแจกแจงแบบเกาส์เซียน)

เพื่อทำการตรวจจับความผิดปกติ คุณจะต้องปรับโมเดลให้เข้ากับการแจกแจงของข้อมูลก่อน

* กำหนดชุดฝึก $\{x^{(1)}, ..., x^{(m)}\}$  คุณต้องการประมาณการแจกแจงแบบเกาส์เซียนสำหรับแต่ละ
คุณลักษณะ $x_i$. 

* นึกถึงการแจกแจงแบบเกาส์เซียนที่กำหนดโดย

   $$ p(x ; \mu,\sigma ^2) = \frac{1}{\sqrt{2 \pi \sigma ^2}}\exp^{ - \frac{(x - \mu)^2}{2 \sigma ^2} }$$

   โดยที่  $\mu$ คือค่าเฉลี่ย และ $\sigma^2$ คือความแปรปรวน
   
* สำหรับแต่ละคุณลักษณะ $i = 1\ldots n$, คุณต้องค้นหาพารามิเตอร์ $\mu_i$ และ  $\sigma_i^2$ ที่เหมาะสมกับข้อมูลในมิติที่ $i$- $\{x_i^{(1)}, ..., x_i^{(m)}\}$ (มิติที่ i ของแต่ละตัวอย่าง)

### 2.3.1 การประมาณค่าพารามิเตอร์สำหรับการแจกแจงแบบเกาส์เซียน

**การใช้งาน:**: 

งานของคุณคือการเติมโค้ดใน `estimate_gaussian` ด้านล่าง

<a name="ex01"></a>
### แบบฝึกหัด 1

โปรดเติมฟังก์ชัน `estimate_gaussian` ด้านล่างเพื่อคำนวณ `mu` (ค่าเฉลี่ยสำหรับแต่ละฟีเจอร์ใน  `X`) และ  `var` (ความแปรปรวนสำหรับแต่ละฟีเจอร์ใน `X`). 

คุณสามารถประมาณค่าพารามิเตอร์ ($\mu_i$, $\sigma_i^2$),  ของฟีเจอร์ที่ $i$-th
โดยใช้สมการต่อไปนี้ เพื่อประมาณค่าเฉลี่ย คุณจะใช้:
$$\mu_i = \frac{1}{m} \sum_{j=1}^m x_i^{(j)}$$

และสำหรับความแปรปรวน คุณจะใช้:

$$\sigma_i^2 = \frac{1}{m} \sum_{j=1}^m (x_i^{(j)} - \mu_i)^2$$

หากคุณติดขัด คุณสามารถตรวจสอบคำแนะนำที่นำเสนอหลังเซลล์ด้านล่างเพื่อช่วยคุณในการใช้งาน

In [ ]:
# UNQ_C1
# GRADED FUNCTION: estimate_gaussian

def estimate_gaussian(X): 
    """
    Calculates mean and variance of all features 
    in the dataset
    
    Args:
        X (ndarray): (m, n) Data matrix
    
    Returns:
        mu (ndarray): (n,) Mean of all features
        var (ndarray): (n,) Variance of all features
    """

    m, n = X.shape
    
    ### START CODE HERE ### 
    
    
    ### END CODE HERE ### 
        
    return mu, var

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
  
   * คุณสามารถนำฟังก์ชันนี้ไปใช้งานได้สองวิธี:
      * 1 - โดยใช้ลูปซ้อนกันสองชั้น - ลูปแรกวนรอบ คอลัมน์  **columns** ของ `X`(แต่ละฟีเจอร์) จากนั้นวนรอบแต่ละจุดข้อมูล
      * 2 - ในลักษณะเวกเตอร์โดยใช้ `np.sum()` กับพารามิเตอร์ `axis = 0`  (เนื่องจากเราต้องการผลรวมสำหรับแต่ละคอลัมน์)

    
   * นี่คือวิธีการโครงสร้างการนำฟังก์ชันนี้ไปใช้งานโดยรวมสำหรับการใช้งานแบบเวกเตอร์:
   
     ```python  
    def estimate_gaussian(X): 
        m, n = X.shape
    
        ### START CODE HERE ### 
        mu = # Your code here to calculate the mean of every feature
        var = # Your code here to calculate the variance of every feature 
        ### END CODE HERE ### 
        
        return mu, var
    ```

    หากคุณยังติดขัด คุณสามารถตรวจสอบคำใบ้ด้านล่างเพื่อหาวิธีคำนวณ `mu` และ  `var`.
    
    <details>
          <summary><font size="2" color="darkblue"><b> คำแนะนำในการคำนวณ mu</b></font></summary>
           &emsp; &emsp; คุณสามารถใช้ <a href="https://numpy.org/doc/stable/reference/generated/numpy.sum.html">np.sum</a> ใช้ `axis = 0` เพื่อหาผลรวมของแต่ละคอลัมน์ของอาร์เรย์
          <details>
              <summary><font size="2" color="blue"><b>&emsp; &emsp; คำแนะนำเพิ่มเติมในการคำนวณ mu</b></font></summary>
               &emsp; &emsp; คุณสามารถคำนวณค่า mu คือ <code>mu = 1 / m * np.sum(X, axis = 0)</code>
           </details>
    </details>
    
    <details>
          <summary><font size="2" color="darkblue"><b> คำแนะนำในการคำนวณ var</b></font></summary>
           &emsp; &emsp; You can use <a href="https://numpy.org/doc/stable/reference/generated/numpy.sum.html">np.sum</a> ใช้ `axis = 0` เพื่อหาผลรวมของแต่ละคอลัมน์ของอาร์เรย์ และ <code>**2</code> รับค่า square.
          <details>
              <summary><font size="2" color="blue"><b>&emsp; &emsp; คำแนะนำเพิ่มเติมในการคำนวณ var</b></font></summary>
               &emsp; &emsp; คุณสามารถคำนวณค่า var คือ <code> var = 1 / m * np.sum((X - mu) ** 2, axis = 0)</code>
           </details>
    </details>
    
</details>

คุณสามารถตรวจสอบว่าการใช้งานของคุณถูกต้องหรือไม่โดยการรันโค้ดทดสอบต่อไปนี้:

In [ ]:
# Estimate mean and variance of each feature
mu, var = estimate_gaussian(X_train)              

print("Mean of each feature:", mu)
print("Variance of each feature:", var)
    
# UNIT TEST
from public_tests_a import *
estimate_gaussian_test(estimate_gaussian)

**ผลลัพธ์คาดหวัง**:
<table>
  <tr>
    <td> <b>Mean of each feature: <b>  </td> 
    <td> [14.11222578 14.99771051]</td> 
   </tr>    
   <tr>
    <td> <b>Variance of each feature: <b>  </td>
     <td> [1.83263141 1.70974533] </td> 
  </tr>
</table>

ตอนนี้คุณได้เรียนรู้วิธีการเขียนโค้ด  `estimate_gaussian`แล้ว เราจะมาสร้างภาพแสดงเส้นโค้งระดับของการแจกแจงแบบเกาส์เซียนที่ฟิตแล้ว

คุณควรได้กราฟที่คล้ายกับรูปด้านล่าง 
<img src="images/figure2_a.png" width="500" height="500">


จากพล็อตของคุณ คุณจะเห็นว่าตัวอย่างส่วนใหญ่จะอยู่ในบริเวณที่มีความน่าจะเป็นสูงสุด ในขณะที่ตัวอย่างผิดปกติจะอยู่ในบริเวณที่มีความน่าจะเป็นต่ำกว่า

In [ ]:
# Returns the density of the multivariate normal
# at each data point (row) of X_train
p = multivariate_gaussian(X_train, mu, var)

#Plotting code 
visualize_fit(X_train, mu, var)

### 2.3.2 การเลือกเกณฑ์ $\epsilon$

ตอนนี้คุณได้ประมาณค่าพารามิเตอร์ของ Gaussian แล้ว คุณสามารถตรวจสอบว่าตัวอย่างใดมีโอกาสสูงมากและตัวอย่างใดมีโอกาสต่่มาก


* ตัวอย่างที่มีโอกาสต่ำมีแนวโน้มที่จะเป็นความผิดปกติในชุดข้อมูลของเรา
* วิธีหนึ่งในการกำหนดตัวอย่างใดเป็นความผิดปกติคือการเลือกเกณฑ์โดยใช้ชุดตรวจสอบแบบ cross-validation 

ในส่วนนี้ คุณจะเติมโค้ดใน  `select_threshold` เพื่อเลือกเกณฑ์ $\varepsilon$ โดยใช้คะแนน $F_1$ บนชุดตรวจสอบแบบ cross-validation

* สำหรับสิ่งนี้ เราจะใช้ชุดตรวจสอบแบบ cross-validation 
$\{(x_{\rm cv}^{(1)}, y_{\rm cv}^{(1)}),\ldots, (x_{\rm cv}^{(m_{\rm cv})}, y_{\rm cv}^{(m_{\rm cv})})\}$, โดยที่ป้ายกำกับ $y=1$ สอดคล้องกับตัวอย่างผิดปกติ และ $y=0$ สอดคล้องกับตัวอย่างปกติ 
* สำหรับแต่ละตัวอย่างการตรวจสอบแบบ cross-validation เราจะคำนวณ  $p(x_{\rm cv}^{(i)})$. เวกเตอร์ของความน่าจะเป็นทั้งหมดเหล่านี้  $p(x_{\rm cv}^{(1)}), \ldots, p(x_{\rm cv}^{(m_{\rm cv})})$ ถูกส่งผ่านไปยัง `select_threshold` ในเวกเตอร์ `p_val`. 
* ป้ายกำกับที่สอดคล้องกัน $y_{\rm cv}^{(1)}, \ldots, y_{\rm cv}^{(m_{\rm cv})}$ ถูกส่งผ่านไปยังฟังก์ชันเดียวกันในเวกเตอร์ `y_val`.

<a name="ex02"></a>
### แบบฝึกหัดที่ 2

โปรดเติมฟังก์ชัน `select_threshold` ด้านล่างเพื่อค้นหาค่า threshold ที่ดีที่สุดในการเลือกค่าผิดปกติตามผลลัพธ์จากชุดตรวจสอบ (`p_val`) และค่าจริง (`y_val`). 

* ในโค้ด  `select_threshold`, ที่ให้มา มีลูปอยู่แล้วที่จะลองค่าต่างๆ ของ $\varepsilon$ หลายค่า และเลือก $\varepsilon$ ที่ดีที่สุดตามคะแนน `F1`

* คุณต้องเขียนโค้ดเพื่อคำนวณคะแนน `F1` จากการเลือก `epsilon` เป็นเกณฑ์และวางค่าใน in `F1`. 

  * โปรดจำไว้ว่าหากตัวอย่าง $x$ มีความน่าจะเป็นต่ำ $p(x) < \varepsilon$ แสดงว่าถูกจัดประเภทเป็นค่าผิดปกติ
        
  * จากนั้น คุณสามารถคำนวณความแม่นยำ (precision) และการเรียกคืน (recall) ได้ดังนี้:
   $$\begin{aligned}
   prec&=&\frac{tp}{tp+fp}\\
   rec&=&\frac{tp}{tp+fn},
   \end{aligned}$$ โดยที่

    * $tp$  คือจำนวน true positive: ป้ายกำกับจริงระบุว่าเป็นค่าผิดปกติ และอัลกอริทึมของเราจัดประเภทถูกต้องว่าเป็นค่าผิดปกติ
    * $fp$ คือจำนวน false positive: ป้ายกำกับจริงระบุว่าไม่ใช่ค่าผิดปกติ แต่อัลกอริทึมของเราจัดประเภทผิดว่าเป็นค่าผิดปกติ
    * $fn$  คือจำนวน false negative: ป้ายกำกับจริงระบุว่าเป็นค่าผิดปกติ แต่อัลกอริทึมของเราจัดประเภทผิดว่าไม่ใช่ค่าผิดปกติ

  * คะแนน  $F_1$ คำนวณโดยใช้ความแม่นยำ ($prec$) และการเรียกคืน ($rec$)  ดังนี้:
  
    $$F_1 = \frac{2\cdot prec \cdot rec}{prec + rec}$$ 

**หมายเหตุการใช้งาน:** 
เพื่อคำนวณ $tp$, $fp$ และ  $fn$, คุณอาจสามารถใช้การนำไปใช้แบบเวกเตอร์แทนการวนลูปผ่านตัวอย่างทั้งหมด


หากคุณติดขัด คุณสามารถตรวจสอบคำแนะนำที่นำเสนอหลังเซลล์ด้านล่างเพื่อช่วยคุณในการใช้งาน

In [ ]:
# UNQ_C2
# GRADED FUNCTION: select_threshold

def select_threshold(y_val, p_val): 
    """
    Finds the best threshold to use for selecting outliers 
    based on the results from a validation set (p_val) 
    and the ground truth (y_val)
    
    Args:
        y_val (ndarray): Ground truth on validation set
        p_val (ndarray): Results on validation set
        
    Returns:
        epsilon (float): Threshold chosen 
        F1 (float):      F1 score by choosing epsilon as threshold
    """ 

    best_epsilon = 0
    best_F1 = 0
    F1 = 0
    
    step_size = (max(p_val) - min(p_val)) / 1000
    
    for epsilon in np.arange(min(p_val), max(p_val), step_size):
    
        ### START CODE HERE ### 
        
        
        
        
        
        
        
        ### END CODE HERE ### 
        
        if F1 > best_F1:
            best_F1 = F1
            best_epsilon = epsilon
        
    return best_epsilon, best_F1

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>

   * ต่อไปนี้คือวิธีการโครงสร้างการนำไปใช้โดยรวมของฟังก์ชันนี้สำหรับการนำไปใช้แบบเวกเตอร์
     ```python  
    def select_threshold(y_val, p_val): 
        best_epsilon = 0
        best_F1 = 0
        F1 = 0
    
        step_size = (max(p_val) - min(p_val)) / 1000
    
        for epsilon in np.arange(min(p_val), max(p_val), step_size):
    
            ### START CODE HERE ### 
            predictions = # Your code here to calculate predictions for each example using epsilon as threshold
        
            tp = # Your code here to calculate number of true positives
            fp = # Your code here to calculate number of false positives
            fn = # Your code here to calculate number of false negatives
        
            prec = # Your code here to calculate precision
            rec = # Your code here to calculate recall
        
            F1 = # Your code here to calculate F1
            ### END CODE HERE ### 
        
            if F1 > best_F1:
                best_F1 = F1
                best_epsilon = epsilon
        
        return best_epsilon, best_F1
    ```

    หากคุณยังติดขัด คุณสามารถตรวจสอบคำใบ้ด้านล่างเพื่อหาคำตอบในการคำนวณตัวแปรแต่ละตัว
    
    <details>
          <summary><font size="2" color="darkblue"><b> คำแนะนำในการคำนวณ predictions</b></font></summary>
           &emsp; &emsp; หากตัวอย่าง  𝑥  มีความน่าจะเป็นต่ำ    $p(x) < \epsilon$ , แสดงว่ามันถูกจัดประเภทเป็นความผิดปกติ เพื่อให้ได้การคาดการณ์สำหรับแต่ละตัวอย่าง   (0/False สำหรับปกติ และ 1/True สำหรับความผิดปกติ) คุณสามารถใช้ <code>predictions = (p_val < epsilon)</code>
    </details>
    
    <details>
          <summary><font size="2" color="darkblue"><b> คำแนะนำในการคำนวณ tp, fp, fn</b></font></summary>
           &emsp; &emsp; 
        <ul>
          <li>หากคุณมีค่าไบนารี่หลายค่าในเวกเตอร์ไบนารี $n$ มิติ คุณสามารถหาจำนวนค่าที่เป็น 0 ในเวกเตอร์นี้: `np.sum(v == 0)` </li>
          <li> คุณสามารถใช้ตัวดำเนินการ and เชิงตรรกะกับเวกเตอร์ไบนารีดังกล่าวได้เช่นกัน ตัวอย่างเช่น  `predictions`เป็นเวกเตอร์ไบนารีที่มีขนาดเท่ากับจำนวนชุดตรวจสอบข้ามของคุณ โดยที่องค์ประกอบที่ $i$ -th  จะเท่ากับ 1 หากอัลกอริทึมของคุณพิจารณา $x_{\rm cv}^{(i)}$ ว่าเป็นค่าผิดปกติ และเป็น 0 มิฉะนั้น </li>
          <li>จากนั้น คุณสามารถคำนวณจำนวนผลบวกลวง (false positives) โดยใช้:  
<code>fp = sum((predictions == 1) & (y_val == 0))</code>.</li>
        </ul>
         <details>
              <summary><font size="2" color="blue"><b>&emsp; &emsp; คำแนะนำเพิ่มเติมในการคำนวณ tp, fn</b></font></summary>
               &emsp; &emsp;
             <ul>
              <li> คุณสามารคำนวณ tp คือ <code> tp = np.sum((predictions == 1) & (y_val == 1))</code></li>
              <li> คุณสามารคำนวณ tn คือ <code> fn = np.sum((predictions == 0) & (y_val == 1))</code></li>  
              </ul>
          </details>
    </details>
        
    <details>
          <summary><font size="2" color="darkblue"><b> คำแนะนำในการคำนวณ precision</b></font></summary>
           &emsp; &emsp; คุณสามารคำนวณ precision คือ <code>prec = tp / (tp + fp)</code>
    </details>
        
    <details>
          <summary><font size="2" color="darkblue"><b> คำแนะนำในการคำนวณ recall</b></font></summary>
           &emsp; &emsp; คุณสามารคำนวณ recall คือ <code>rec = tp / (tp + fn)</code>
    </details>
        
    <details>
          <summary><font size="2" color="darkblue"><b>คำแนะนำในการคำนวณ F1</b></font></summary>
           &emsp; &emsp; คุณสามารคำนวณ F1 คือ <code>F1 = 2 * prec * rec / (prec + rec)</code>
    </details>
    
</details>

คุณสามารถตรวจสอบการใช้งานโค้ดของคุณโดยใช้โค้ดด้านล่างนี้

In [ ]:
p_val = multivariate_gaussian(X_val, mu, var)
epsilon, F1 = select_threshold(y_val, p_val)

print('Best epsilon found using cross-validation: %e' % epsilon)
print('Best F1 on Cross Validation Set: %f' % F1)
    
# UNIT TEST
select_threshold_test(select_threshold)


**ผลลัพธ์คาดหวัง**:
<table>
  <tr>
    <td> <b>Best epsilon found using cross-validation: <b>  </td> 
    <td> 8.99e-05</td> 
   </tr>    
   <tr>
    <td> <b>Best F1 on Cross Validation Set: <b>  </td>
     <td> 0.875 </td> 
  </tr>
</table>

Now we will run your anomaly detection code and circle the anomalies in the plot (Figure 3 below).

<img src="images/figure3_a.png" width="500" height="500">

In [ ]:
# Find the outliers in the training set 
outliers = p < epsilon

# Visualize the fit
visualize_fit(X_train, mu, var)

# Draw a red circle around those outliers
plt.plot(X_train[outliers, 0], X_train[outliers, 1], 'ro',
         markersize= 10,markerfacecolor='none', markeredgewidth=2)

<a name="2.4"></a>
### 2.4 High dimensional dataset

ตอนนี้ เราจะรันอัลกอริทึมการตรวจจับความผิดปกติที่คุณได้นำไปใช้กับชุดข้อมูลจริงและยากกว่า

ในชุดข้อมูลนี้ แต่ละตัวอย่างถูกอธิบายโดย 11 คุณลักษณะ จับภาพคุณสมบัติมากขึ้นของเซิร์ฟเวอร์คอมพิวเตอร์ของคุณ

เริ่มกันด้วยการโหลดชุดข้อมูล

- ฟังก์ชัน  `load_data()` ที่แสดงด้านล่างจะโหลดข้อมูลลงในตัวแปร `X_train_high`, `X_val_high` และ  `y_val_high`
    -  `_high` หมายถึงการแยกแยะตัวแปรเหล่านี้จากตัวแปรที่ใช้ในส่วนก่อนหน้า
    - เราจะใช้ `X_train_high` เพื่อปรับฟังก์ชันการแจกแจงแบบเกาส์เซียน (Gaussian distribution) 
    - เราจะใช้ `X_val_high` และ  `y_val_high`  เป็นชุดตรวจสอบข้ามเพื่อเลือกเกณฑ์และกำหนดตัวอย่างผิดปกติและปกติ

In [ ]:
# load the dataset
X_train_high, X_val_high, y_val_high = load_data_multi()

#### ตรวจสอบมิติของตัวแปรของคุณ

มาตรวจสอบมิติของตัวแปรใหม่เหล่านี้เพื่อทำความคุ้นเคยกับข้อมูลกัน

In [ ]:
print ('The shape of X_train_high is:', X_train_high.shape)
print ('The shape of X_val_high is:', X_val_high.shape)
print ('The shape of y_val_high is: ', y_val_high.shape)

#### การตรวจจับความผิดปกติ 

ตอนนี้ ลองใช้อัลกอริทึมการตรวจจับความผิดปกติกับชุดข้อมูลใหม่นี้

โค้ดด้านล่างจะใช้โค้ดของคุณเพื่อ
* ประมาณค่าพารามิเตอร์ Gaussian ($\mu_i$ และ $\sigma_i^2$)
* ประเมินความน่าจะเป็นสำหรับทั้งข้อมูลฝึก `X_train_high` ที่คุณประมาณค่าพารามิเตอร์ Gaussian และสำหรับชุดตรวจสอบข้าม `X_val_high`. 
* สุดท้าย จะใช้ `select_threshold` เพื่อหาค่า threshold  $\varepsilon$ ที่ดีที่สุด

In [ ]:
# Apply the same steps to the larger dataset

# Estimate the Gaussian parameters
mu_high, var_high = estimate_gaussian(X_train_high)

# Evaluate the probabilites for the training set
p_high = multivariate_gaussian(X_train_high, mu_high, var_high)

# Evaluate the probabilites for the cross validation set
p_val_high = multivariate_gaussian(X_val_high, mu_high, var_high)

# Find the best threshold
epsilon_high, F1_high = select_threshold(y_val_high, p_val_high)

print('Best epsilon found using cross-validation: %e'% epsilon_high)
print('Best F1 on Cross Validation Set:  %f'% F1_high)
print('# Anomalies found: %d'% sum(p_high < epsilon_high))

**ผลลัพธ์คาดหวัง**:
<table>
  <tr>
    <td> <b>Best epsilon found using cross-validation: <b>  </td> 
    <td> 1.38e-18</td> 
   </tr>    
   <tr>
    <td> <b>Best F1 on Cross Validation Set: <b>  </td>
     <td> 0.615385 </td> 
  </tr>
    <tr>
    <td> <b># anomalies found: <b>  </td>
     <td>  117 </td> 
  </tr>
</table>